<a href="https://colab.research.google.com/github/iamsauravsharma/insincere-question-classification/blob/bishal/lstm%2Bgru_in_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'embeddings', 'test.csv']


In [0]:
df = pd.read_csv("../input/train.csv")

In [0]:
emb_file = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
#emb_file_par =  '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'

In [0]:
from tqdm import tqdm
from tqdm import tqdm_notebook




In [0]:

def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
sentences = df["question_text"].apply(lambda x: x.split()).values
vocab = build_vocab(sentences)

100%|██████████| 1306122/1306122 [00:05<00:00, 230639.07it/s]


In [0]:
def load_embed(file):
    embeddings_index = {}
    f = open(file)
    #f = open('../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt',encoding="utf8", errors='ignore')
    for line in tqdm(f):
        values = line.split(" ")
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()        
    return embeddings_index

In [0]:
glove_dic = load_embed(emb_file)
#glove_dic1=  load_embed(emb_file_par)

2196017it [03:21, 10882.61it/s]


In [0]:
n_words = len(vocab)+1
del vocab
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

train, val = train_test_split(df, test_size=0.1, random_state=42)


tokenizer = Tokenizer(num_words=n_words)
tokenizer.fit_on_texts(list(train.question_text))

q_train = tokenizer.texts_to_sequences(train.question_text)
q_val = tokenizer.texts_to_sequences(val.question_text)
#q_test = tokenizer.texts_to_sequences(df_test.question_text)

max_len = 65
q_train = pad_sequences(q_train,maxlen=max_len)
q_val = pad_sequences(q_val,maxlen=max_len)
#q_test = pad_sequences(q_test,maxlen=max_len)

y_train = train.target
y_val = val.target

del train,val
word_index = tokenizer.word_index
emb_size = glove_dic['.'].shape[0]
emb_matrix = np.zeros((n_words,emb_size))
for w,index in word_index.items():
    if index >= n_words:
        
        continue
    vec = glove_dic.get(w)
    if vec is not None:
        
        emb_matrix[index,:] = vec
        continue
    w = w.lower()
    vec = glove_dic.get(w)
    if vec is not None:
        
        emb_matrix[index,:] = vec
        continue
    w = w.upper()
    vec = glove_dic.get(w)
    if vec is not None:
        
        emb_matrix[index,:] = vec
        continue

    w = w.capitalize()
    vec = glove_dic.get(w)
    if vec is not None:
        
        emb_matrix[index,:] = vec
        continue
        

Using TensorFlow backend.


In [0]:
del glove_dic

In [0]:

emb_matrix_par = np.zeros((n_words,emb_size))
for w,index in word_index.items():
    if index >= n_words:
        
        continue
    vec = glove_dic1.get(w)
    if vec is not None:
        
        emb_matrix_par[index,:] = vec
        continue
    w = w.lower()
    vec = glove_dic1.get(w)
    if vec is not None:
        
        emb_matrix_par[index,:] = vec
        continue
    w = w.upper()
    vec = glove_dic1.get(w)
    if vec is not None:
        
        emb_matrix_par[index,:] = vec
        continue

    w = w.capitalize()
    vec = glove_dic1.get(w)
    if vec is not None:
        
        emb_matrix_par[index,:] = vec
        continue
del glove_dic1


In [0]:
embedding_matrix_com = np.concatenate((emb_matrix, emb_matrix_par), axis=1)
del emb_matrix
del emb_matrix_par

In [0]:
from keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#find the best threshold
def optim_thres(y_val,y_pred):
    score = 0
    thresholds = np.arange(0.1,0.501,0.01)
    for thres in thresholds:
        thres = np.round(thres,2)
        temp_pred = (y_pred > thres).astype(int)
        temp_score = f1_score(y_val,temp_pred)
        print("Thres: {} --------- F1: {}".format(thres,temp_score))
        if temp_score > score:
            score = temp_score
            final_thres = thres
    return final_thres

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import gensim as gn
from tqdm import tqdm_notebook

from keras.layers import LSTM,Bidirectional,TimeDistributed, Embedding,Dense,Input,GlobalMaxPool1D,Flatten,Dropout
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential,Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam,Adamax
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)
        
    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True
        
    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)
    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import gensim as gn
from tqdm import tqdm_notebook

from keras.layers import LSTM,Bidirectional,TimeDistributed, Embedding,Dense,Input,GlobalMaxPool1D,Flatten,Dropout
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential,Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.optimizers import Adam,Adamax
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import LSTM,GRU,Bidirectional,TimeDistributed,Concatenate, Embedding,Dense,Input,GlobalMaxPool1D,Flatten,Dropout,Conv1D,SpatialDropout1D,GlobalAveragePooling1D
from keras.layers import CuDNNLSTM,CuDNNGRU
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding,Flatten,Dense
from keras.optimizers import Adam,Adamax
from keras.engine.topology import Layer
from keras.layers.normalization import BatchNormalization


In [0]:
emb_size=300
inp = Input(shape=(max_len,))
x = Embedding(input_dim=n_words,output_dim=emb_size, weights=[emb_matrix])(inp)
#x = Embedding(input_dim=n_words,output_dim=embed_size, weights=[embedding_matrix])(inp)
x1 = Bidirectional(CuDNNLSTM(100, return_sequences=True))(x)
x2 = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
x1 = Conv1D(50,10,activation='relu')(x1)
x1 = GlobalMaxPool1D()(x)
x2 = Attention(step_dim=max_len)(x2)
conc = Concatenate()([x1, x2])
x = Dense(128,activation='relu')(conc)
x = Dropout(0.1)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=inp,output=x)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 65)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 65, 300)      152647200   input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 65, 200)      241200      embedding_1[0][0]                
__________________________________________________________________________________________________
global_max_pooling1d_

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [0]:
model_name = 'lstm_glove_emb'
checkpoint = ModelCheckpoint(filepath='./{}.hdf5'.format(model_name),
                             monitor='val_f1',mode='max',verbose=1,
                            save_best_only=True)

adamax= Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['acc',f1])
history  = model.fit(q_train,y_train,batch_size=1500,epochs=7,validation_data=(q_val,y_val),verbose=1,callbacks=[checkpoint])

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 152647200 elements. This may consume a large amount of memory.
  num_elements)


Train on 1175509 samples, validate on 130613 samples
Epoch 1/7
1175509/1175509 [==============================] - 99s 84us/step - loss: 0.1264 - acc: 0.9515 - f1: 0.5205 - val_loss: 0.1040 - val_acc: 0.9584 - val_f1: 0.5949

Epoch 00001: val_f1 improved from -inf to 0.59491, saving model to ./lstm_glove_emb.hdf5
Epoch 2/7
1175509/1175509 [==============================] - 89s 75us/step - loss: 0.1026 - acc: 0.9588 - f1: 0.6350 - val_loss: 0.0989 - val_acc: 0.9602 - val_f1: 0.6534

Epoch 00002: val_f1 improved from 0.59491 to 0.65335, saving model to ./lstm_glove_emb.hdf5
Epoch 3/7
1175509/1175509 [==============================] - 89s 76us/step - loss: 0.0957 - acc: 0.9614 - f1: 0.6638 - val_loss: 0.0977 - val_acc: 0.9609 - val_f1: 0.6571

Epoch 00003: val_f1 improved from 0.65335 to 0.65706, saving model to ./lstm_glove_emb.hdf5
Epoch 4/7
1175509/1175509 [==============================] - 89s 76us/step - loss: 0.0899 - acc: 0.9635 - f1: 0.6873 - val_loss: 0.0972 - val_acc: 0.9609 - va